# String the attribute

In [7]:
class IntegerValue:
    def __set__(self, instance, value):
        instance.stored_value = int(value)

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return instance.stored_value


In [8]:
IntegerValue()

In [19]:
class Point1D:
    x = IntegerValue()

In [20]:
p1 = Point1D()
p2 = Point1D()

In [21]:
p1.x = 100.2
p2.x = 200.4

In [22]:
p1.x, p2.x

(100, 200)

In [23]:
p1.__dict__, p2.__dict__

({'stored_value': 100}, {'stored_value': 200})

In [24]:
class Point2D:
    x = IntegerValue()
    y = IntegerValue()

In [25]:
p1 = Point2D()
p2 = Point2D()

In [26]:
p1.x = 100.2
p1.y = 200.2

In [27]:
p1.__dict__

{'stored_value': 200}

Both x,y are stored in the same key , since we hardcode the key

In [28]:
class IntegerValue:
    def __init__(self, name):
        self.storage_name = "_" + name

    def __set__(self, instance, value):
        setattr(instance, self.storage_name, value)

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return getattr(instance, self.storage_name)


In [39]:
IntegerValue("1")

In [30]:
class Point2D:
    x = IntegerValue("x")
    y = IntegerValue("y")

In [31]:
p1 = Point2D()

In [35]:
p1.x = 100.2
p1.y = 220.2

In [36]:
p1.__dict__

{'_x': 100.2, '_y': 220.2}

In [37]:
p1.x, p1.y

(100.2, 220.2)

So this approach can work just fine, but there are a few drawbacks:

1. The user needs to specify the name of the property twice
2. We assume that `_ + name` is not also used by the class in which the descriptor exists (so that could be a major problem)
3. We assume we can add an attribute to the instance - but what if it uses slots?

One way we could get around each of those problems is by using the descriptor instance itself to store the instance values. But as we saw earlier, we can't just set an attribute in the descriptor instance, since that would be shared across multiple instances of the class containing the descriptor.

# descriptor dict

In [48]:
class IntegerValue:
    def __init__(self):
        self.values = {}

    def __set__(self, instance, value):
        self.values[instance] = int(value)

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return self.values.get(instance)


In [49]:
class Point2D:
    x = IntegerValue()
    y = IntegerValue()

In [50]:
p1 = Point2D()
p2 = Point2D()

In [57]:
p1.x = 100.2
p1.y = 200.2

In [58]:
p1.__dict__, hex(id(p1))

({}, '0x1b2bb282550')

In [59]:
Point2D.x.values

{<__main__.Point2D at 0x1b2bb282550>: 100,
 <__main__.Point2D at 0x1b2baf45390>: 200}

In [60]:
Point2D.y.values

{<__main__.Point2D at 0x1b2bb282550>: 200,
 <__main__.Point2D at 0x1b2baf45390>: 200}

In [61]:
p2.x = 2000.2
p2.y = 1000.2

In [64]:
p2.x, p2.y, hex(id(p2))

(2000, 1000, '0x1b2baf45390')

In [63]:
Point2D.x.values

{<__main__.Point2D at 0x1b2bb282550>: 100,
 <__main__.Point2D at 0x1b2baf45390>: 2000}

In [66]:
Point2D.y.values

{<__main__.Point2D at 0x1b2bb282550>: 200,
 <__main__.Point2D at 0x1b2baf45390>: 1000}

In [67]:
p1.x, p1.y, p2.x, p2.y

(100, 200, 2000, 1000)

We actually have a potential memory leak - notice how the dictionary in the descriptor instance is also storing a reference to the point object - as a key in the dictionary

In [1]:
import ctypes

In [2]:
class IntegerValue:
    def __init__(self):
        self.values = {}

    def __set__(self, instance, value):
        self.values[instance] = int(value)

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return self.values.get(instance)

class Point2D:
    x = IntegerValue()
    y = IntegerValue()

In [3]:
def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [5]:
p1 = Point2D()
id_p1 = id(p1)

In [6]:
ref_count(id_p1)

1

In [7]:
p1.x = 100.2

In [9]:
ref_count(id_p1)

4

In [10]:
del p1

In [11]:
ref_count(id_p1)

6